## Proyecto final procesamiento de lenguaje natural

**Integrantes:**

* Laura del Pilar Torres Toro - Código: 617202012
* Gabriel Cruz Parra - Código: 617202013

**Planteamiento del Problema:**

* Determinar la variación de opinión de x personajes políticos en los últimos 10 años respecto a un tema determinado.

**Motivación:**

* Determinar si en la clase política colombiana existe una tendencia a contradecir sus convicciones.

**Metodología:**

* Adquisición de los datos 
* Pre procesamiento (palabras vacías, caracteres especiales, traducción y agregación por año)
* Clasificación (análisis de sentimientos)
* Determinación de variación
* Clustering
* Word Cloud
* LDA

In [1]:
import snscrape.modules.twitter as sntwitter
from datetime import datetime
from datetime import timedelta
import pandas as pd
import numpy as np
import re
import six
import ipywidgets as widgets
from google.cloud import translate_v2 as translate
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from textblob import TextBlob 
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from flair.models import TextClassifier
from flair.data import Sentence
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import silhouette_score
from gensim.corpora import Dictionary
import pyLDAvis.gensim_models
from gensim.models import LdaModel
from wordcloud import WordCloud, ImageColorGenerator
import PIL.Image
from IPython.display import display



import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
init_notebook_mode(connected=True)

vader = SentimentIntensityAnalyzer()
translate_client = translate.Client()
stopwords_sp = stopwords.words('spanish')
classifier = TextClassifier.load('sentiment')
#cambiar el valor de la constante para cargar archivos
RUTA_ARCHIVOS = ""

2021-06-03 01:19:22,923 loading file C:\Users\Usuario\.flair\models\sentiment-en-mix-distillbert_4.pt


In [2]:
listadoPoliticos = pd.read_excel(f'{RUTA_ARCHIVOS}listadoPoliticos.xlsx')

listadoTemas = pd.read_excel(f'{RUTA_ARCHIVOS}listadoPoliticos.xlsx', sheet_name='Temas')

C:\Users\Usuario\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [3]:
opcionesPoliticos= []
for index, row in listadoPoliticos.iterrows():
    opcionesPoliticos.append((row["Politico"]+" - "+row["Cargo"], row["Cuenta"]))

opcionesPoliticos = sorted(opcionesPoliticos)

opcionesTemas = []
for index, row in listadoTemas.iterrows():
    opcionesTemas.append(row["Tema"])

opcionesTemas = sorted(opcionesTemas)


C:\Users\Usuario\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [4]:
# Función de recuperación de tweets
def get_tweet_dataset(maxTweets):
    politico = dropdown_politicos.value
    tema = dropdown_temas.value
    now = datetime.now()
    retro_search = now - timedelta(days = (365*text_vigencias.value))
    hoy = now.strftime("%Y-%m-%d")
    pasado = retro_search.strftime("%Y-%m-%d")
    tweet_list = []

    for i,tweet in enumerate(sntwitter.TwitterSearchScraper(f'{tema} since:{pasado} until:{hoy} from:{politico}').get_items()) :
        if i > maxTweets :
            break
            
        tweet_list.append([tweet.date, tweet.id, tweet.content, tweet.user.username, tweet.user.displayname, tweet])
    
    # Creating a dataframe from the tweets list above 
    tweets_df = pd.DataFrame(tweet_list, columns=['Fecha', 'tweet_id', 'contenido', 'usuario', 'usuario_mostrar', 'enlace'])
    return tweets_df 

C:\Users\Usuario\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [5]:
# Definición de función de pre-procesamiento
def preProcesado(texto):
    pattern1 = '?P<pic>pic.twitter.com/[^\s]+'
    pattern2 = '?P<url>https?://[^\s]+'
    
    links = [tuple(j for j in i if j)[-1] for i in re.findall(f"({pattern1})|({pattern2})",texto)]
    for link in links:
        texto = texto.replace(link,"")
    
    hashtags = [interaction for interaction in texto.split() if interaction.startswith("#")]
    for hashtag in hashtags:
        texto = texto.replace(hashtag, "")
        
    mentions = [interaction for interaction in texto.split() if interaction.startswith("@")]
    for mention in mentions:
        texto = texto.replace(mention, "")
    
    texto = texto.lower()
    #texto = re.sub(r"[\W\d_]+", " ", texto)
    texto = texto.split() # Tokenizar
    #texto = [palabra for palabra in texto if palabra not in stopwords_sp]
    #texto = [palabra for palabra in texto if len(palabra)>=2]
    texto = " ".join(texto)
    if len(texto.replace(" ","")) <1:
        texto = "Tweet sin contenido"
        
    return texto

#función de traducción
def traducir(texto, lenguaje):
    if isinstance(texto, six.binary_type):
        texto = texto.decode("utf-8")
    # Text can also be a sequence of strings, in which case this method
    # will return a sequence of results for each text.
    resultado = translate_client.translate(texto, target_language=lenguaje)
    return resultado['translatedText']

# función de aplicación de pre precesado
def aplicar_preprocesado(frame):
    frame['pre_procesado'] = frame['contenido'].apply(lambda x: preProcesado(x))
    frame['pre_procesado'] = frame['pre_procesado'].fillna("Tweet sin contenido")
    lenguaje = 'en'
    frame['traducido'] = frame['pre_procesado'].apply(lambda x: traducir(x,lenguaje))
    frame['Año'] = frame['Fecha'].dt.year
    return frame

# realización de análisis de sentimientos para cada tweet flair
def sent_flair(texto):
    sentence = Sentence(texto)
    classifier.predict(sentence)
    result = sentence.labels[0]
    label = result.value
    score = result.score
    if label == 'POSITIVE':
        return score
    if label == 'NEGATIVE':
        return -1 * score
    return score

# función aplicación de análisis de sentimientos
def aplicar_sentimiento(frame):
    frame['textblob_sentiment_en'] = frame['traducido'].apply(lambda x: TextBlob(x).sentiment.polarity)
    frame['vader_sentiment_en'] = frame['traducido'].apply(lambda x: vader.polarity_scores(x)['compound'])
    frame['flair_sentiment_en'] = frame['traducido'].apply(lambda x: sent_flair(x))
    return frame

#Definición de funciones de filtrado
def get_positivos(frame, columna, agrupador):
    filtro = frame[frame[columna]>0.0]
    return filtro.groupby([agrupador]).count()

def get_negativos(frame, columna, agrupador):
    filtro = frame[frame[columna]<0.0]
    return filtro.groupby([agrupador]).count()

def get_neutros(frame, columna, agrupador):
    filtro = frame[frame[columna]==0.0]
    return filtro.groupby([agrupador]).count()


# Función de obtención de resumen de tweets por naturaleza
def get_resumen(conteoPositivosFlairEn, conteoNegativosFlairEn, conteoNeutrosFlairEn,
                        conteoPositivosVaderEn, conteoNegativosVaderEn, conteoNeutrosVaderEn,
                        conteoPositivosTextblobEn, conteoNegativosTextblobEn, conteoNeutrosTextblobEn):
    conteosData = pd.concat([conteoPositivosFlairEn, conteoNegativosFlairEn, conteoNeutrosFlairEn,
                        conteoPositivosVaderEn, conteoNegativosVaderEn, conteoNeutrosVaderEn,
                        conteoPositivosTextblobEn, conteoNegativosTextblobEn, conteoNeutrosTextblobEn], axis= 1)

    listaValores = ['Año','+ flair(en)', '- flair(en)', '0 flair(en)',
               '+ vader(en)', '- vader(en)', '0 vader(en)',
               '+ textblob(en)', '- textblob(en)', '0 textblob(en)']
    conteosData =pd.DataFrame(conteosData).reset_index()

    conteosData.columns = listaValores
    conteosData = conteosData.fillna(0)

    return conteosData

def get_scatter(frame, politico, tema):
    fig = px.scatter(frame, y="Atributo", x="Año", color="Atributo", size="Cantidad",
                  title=f'Análisis tweets usuario: {politico}; tema: {tema}')
    return fig

# definir otra forma de obtener resumen
def get_resumen_por_año(frame):
    dataGraph = frame.T
    dataGraph = dataGraph.reset_index()
    dataGraph.loc[0:0,"index"] = 'Atributo'
    headers = dataGraph.iloc[0].astype(str)
    dataGraph  = pd.DataFrame(dataGraph.values[1:], columns=headers)
    #auxiliar = dataGraph.iloc[0:,1:]
    #headers  = auxiliar.columns.to_list()
    return dataGraph

def get_best_k(x):
    sil = []
    kmax = 15
    # dissimilarity would not be defined for a single cluster, thus, minimum number of clusters should be 2
    for k in range(2, kmax+1):
      kmeans = KMeans(n_clusters = k).fit(x)
      labels = kmeans.labels_
      # obtenemos el silhouette score  con sklearn
      sil.append(silhouette_score(x, labels, metric = 'euclidean'))
    
    if round((max(sil)*100), 0) <= 2 or round((max(sil)*100), 0) > 8 :
        best = 3
    else:
        best = int(round((max(sil)*100), 0))
    return best  

# función para obtener la gráfica de cluster
def get_cluster(frame):
    
    # Matriz TFIDF
    tfidf_vect = TfidfVectorizer(preprocessor=preProcesado)
    tfidf = tfidf_vect.fit_transform(frame.contenido.values)
    matrix = pd.DataFrame(tfidf.toarray(), columns = tfidf_vect.get_feature_names())
    
    # Obtener el mejor k, si es menor o igual a 2 entonces se dejan 3 grupos
    k= get_best_k(matrix)
    
    # se realiza entrenamiento del modelo con el k obtenido
    model = KMeans(n_clusters=k)
    model.fit(matrix)
    # asignando el número de cluster a columna en el frame de datos
    frame['cluster'] = model.labels_

    # cálculo de vecinos y centroides
    nbrs = NearestNeighbors(n_neighbors=3, metric='euclidean').fit(matrix.values)
    clust_cnt = frame['cluster'].value_counts()
    clust_cnt_pct = frame['cluster'].value_counts(normalize=True)
    centroids = model.cluster_centers_
    terms = tfidf_vect.get_feature_names()
    order_centroids = centroids.argsort()[:, ::-1]
    
    # Nombrar clusteres en columna del frame de datos
    frame['nombres_clusters'] = frame['cluster'].apply(lambda val: "cluster: "+str(val))
    
    # Análisis de componentes principales o reducción de la dimensionalidad
    pca = PCA(n_components=2)
    result = pca.fit_transform(matrix)
    result = pd.DataFrame(result)
    result.columns = ['X', 'Y']
    result['cluster'] = frame.nombres_clusters.values
    result['texto'] = frame.contenido.apply(lambda val: val[:140])
    
    # Construcción de la gráfica de clústeres
    trace = px.scatter(result, y="Y", x="X", color="cluster", hover_name=result['texto'], title=f'Análisis cluster')
    return trace
    


#Modelo de LDA
def pre_procesado(texto):  
    texto = texto.lower()
    texto = re.sub(r"[\W\d]+", " ", texto)
    texto = texto.split() # tokenización 
    texto = [palabra for palabra in texto if palabra not in stopwords_sp]
    return texto 


def get_lda(frame):
    frame['pre_procesado2'] = frame['pre_procesado'].apply(lambda val: pre_procesado(val))
    # Crear una represantación de los documentos en forma de diccionario
    dictionary = Dictionary(frame['pre_procesado2'].values)
    # Filtrar palabras muy frecuentes o infrecuentes
    dictionary.filter_extremes(no_below=2, no_above=0.7) 
    corpus = [dictionary.doc2bow(text) for text in frame['pre_procesado2'].values]
    model = LdaModel(corpus = corpus, id2word = dictionary, num_topics = 7, passes=10)

    lda_display = pyLDAvis.gensim_models.prepare(model, corpus, dictionary)
    return lda_display

def get_word_cloud(frame, columna):
    palabras = frame[columna].str.cat(sep=" ")
    img = PIL.Image.open(f'{RUTA_ARCHIVOS}Mapa13.jpg')
    mask = np.array(img)
    wordcloud = WordCloud(width=800, height=800,
                      max_font_size=150, max_words=200,
                      background_color="white", colormap="ocean_r",
                      stopwords=stopwords_sp, 
                      collocations=True,
                      mask=mask, contour_width=1, contour_color='blue').generate(palabras) # https://matplotlib.org/3.1.0/tutorials/colors/colormaps.html
    return wordcloud
    



C:\Users\Usuario\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.

<>:3: DeprecationWarning:

invalid escape sequence \s

<>:4: DeprecationWarning:

invalid escape sequence \s

<>:3: DeprecationWarning:

invalid escape sequence \s

<>:4: DeprecationWarning:

invalid escape sequence \s

<ipython-input-5-933cf10d9c41>:3: DeprecationWarning:

invalid escape sequence \s

<ipython-input-5-933cf10d9c41>:4: DeprecationWarning:

invalid escape sequence \s



### Desarrollo del proyecto

* Obtenemos un conjunto de parámetros deseados por el usuario y ejecutamos el análisis:

In [6]:
# Definir componentes gráficos para los parámetros    
dropdown_politicos = widgets.Dropdown(options =opcionesPoliticos, description="Político: ", layout=widgets.Layout(width='50%'))
dropdown_temas = widgets.Dropdown(options =opcionesTemas, description="Temas: ", layout=widgets.Layout(width='50%'))
slider_vigencias = widgets.IntSlider(description="Años: ", min=1, max=10, step=1, value=5, layout=widgets.Layout(width='50%'))
text_vigencias = widgets.IntText(description="Años: ")
text_max_tweets = widgets.IntText(description="Máx tweets: ", value=1000, min=100, max=5000, step=100,
                                  layout=widgets.Layout(width='50%'))
btn = widgets.Button(description='Procesar')

# Sincroniza el valor del slider con un campo de texto
widgets.jslink((slider_vigencias, 'value'), (text_vigencias, 'value'))

# incluir componentes gráficos en un contenedor horizontal
input_widgets = widgets.HBox([dropdown_politicos, dropdown_temas])
display(input_widgets)

input2_widgets = widgets.HBox([slider_vigencias, text_max_tweets])
display(input2_widgets)

button_widgets = widgets.HBox([btn])
display(button_widgets)

# inicializar variables que intervienen en el proceso
recuperados = None
preprocesados = None
sentimientos = None
resumen = None
conteoPositivosFlairEn = None
conteoNegativosFlairEn = None
conteoNeutrosFlairEn = None
conteoPositivosVaderEn = None
conteoNegativosVaderEn = None
conteoNeutrosVaderEn = None
conteoPositivosTextblobEn = None
conteoNegativosTextblobEn = None
conteoNeutrosTextblobEn = None

# definición de las salidas
output_df = widgets.Output()
output_pre = widgets.Output()
output_sen = widgets.Output()
output_res =widgets.Output()
plot_output = widgets.Output()
cluster_output = widgets.Output()
cloud_output = widgets.Output()
lda_output = widgets.Output()

# Manejador del evento de presionado del botón
def btn_eventhandler(obj):
    # limpiar las salidas de la pantalla (tabs)
    output_df.clear_output()
    output_pre.clear_output()
    output_sen.clear_output()
    output_res.clear_output()
    plot_output.clear_output()
    cluster_output.clear_output()
    lda_output.clear_output()
    cloud_output.clear_output()
    # Mapeo la lógica de cada salida ( que debe pintar en cada tab)
    with output_df:
        recuperados = get_tweet_dataset(text_max_tweets.value)
        display(recuperados)
    with output_pre: 
        if(len(recuperados)>2):
            preprocesados = aplicar_preprocesado(recuperados)
            display(preprocesados[["Año","contenido","pre_procesado","traducido"]])
        else:   
            display(print("Lo sentimos no hemos encontrado suficientes tweets para ejecutar el análisis"))
    with output_sen:
        if(len(recuperados)>2):
            sentimientos = aplicar_sentimiento(preprocesados)
            display(sentimientos[["Año","traducido","flair_sentiment_en","vader_sentiment_en","textblob_sentiment_en"]])
        else:
            display(print("Lo sentimos no hemos encontrado suficientes tweets para ejecutar el análisis"))
    with output_res:
        if (len(recuperados)>2):
            # conteos flair inglés
            conteoPositivosFlairEn = get_positivos(sentimientos[['flair_sentiment_en','Año']],'flair_sentiment_en','Año')
            conteoNegativosFlairEn = get_negativos(sentimientos[['flair_sentiment_en','Año']],'flair_sentiment_en', 'Año')
            conteoNeutrosFlairEn = get_neutros(sentimientos[['flair_sentiment_en','Año']],'flair_sentiment_en','Año')

            # conteos vader inglés
            conteoPositivosVaderEn = get_positivos(sentimientos[['vader_sentiment_en','Año']],'vader_sentiment_en','Año')
            conteoNegativosVaderEn = get_negativos(sentimientos[['vader_sentiment_en','Año']],'vader_sentiment_en','Año')
            conteoNeutrosVaderEn = get_neutros(sentimientos[['vader_sentiment_en','Año']],'vader_sentiment_en','Año')

            # conteos textblob inglés
            conteoPositivosTextblobEn = get_positivos(sentimientos[['textblob_sentiment_en','Año']],'textblob_sentiment_en','Año')
            conteoNegativosTextblobEn = get_negativos(sentimientos[['textblob_sentiment_en','Año']],'textblob_sentiment_en','Año')
            conteoNeutrosTextblobEn = get_neutros(sentimientos[['textblob_sentiment_en','Año']],'textblob_sentiment_en','Año')
        
            resumen= get_resumen(conteoPositivosFlairEn, conteoNegativosFlairEn, conteoNeutrosFlairEn,
                        conteoPositivosVaderEn, conteoNegativosVaderEn, conteoNeutrosVaderEn,
                        conteoPositivosTextblobEn, conteoNegativosTextblobEn, conteoNeutrosTextblobEn)
            display(resumen)
        else:
            display(print("Lo sentimos no hemos encontrado suficientes tweets para ejecutar el análisis"))
    with plot_output:
        if(len(recuperados)>2):
            # conteos flair inglés
            conteoPositivosFlairEn = conteoPositivosFlairEn.reset_index()
            conteoPositivosFlairEn["Atributo"] = "+ flair(en)"
            conteoNegativosFlairEn = conteoNegativosFlairEn.reset_index()
            conteoNegativosFlairEn["Atributo"] = "- flair(en)"
            conteoNeutrosFlairEn = conteoNeutrosFlairEn.reset_index()
            conteoNeutrosFlairEn["Atributo"] = "0 flair(en)"
        
            flair_df = pd.concat([conteoPositivosFlairEn, conteoNegativosFlairEn, conteoNeutrosFlairEn], axis= 0)
            flair_df.columns= ['Año', 'Cantidad', 'Atributo']

            # conteos vader inglés
            conteoPositivosVaderEn = conteoPositivosVaderEn.reset_index()
            conteoPositivosVaderEn["Atributo"] = "+ vader(en)"
            conteoNegativosVaderEn = conteoNegativosVaderEn.reset_index()
            conteoNegativosVaderEn["Atributo"] = "- vader(en)"
            conteoNeutrosVaderEn = conteoNeutrosVaderEn.reset_index()
            conteoNeutrosVaderEn["Atributo"] = "0 vader(en)"
        
            vader_df = pd.concat([conteoPositivosVaderEn, conteoNegativosVaderEn, conteoNeutrosVaderEn], axis= 0)
            vader_df.columns= ['Año', 'Cantidad', 'Atributo']

            # conteos textblob inglés
            conteoPositivosTextblobEn = conteoPositivosTextblobEn.reset_index()
            conteoPositivosTextblobEn["Atributo"] = "+ textblob(en)"
            conteoNegativosTextblobEn = conteoNegativosTextblobEn.reset_index()
            conteoNegativosTextblobEn["Atributo"] = "- textblob(en)"
            conteoNeutrosTextblobEn = conteoNeutrosTextblobEn.reset_index()
            conteoNeutrosTextblobEn["Atributo"] = "0 textblob(en)"
        
            textblob_df = pd.concat([conteoPositivosTextblobEn, conteoNegativosTextblobEn, conteoNeutrosTextblobEn], axis= 0)
            textblob_df.columns= ['Año', 'Cantidad', 'Atributo']
        
            c_df = pd.concat([flair_df, vader_df, textblob_df], axis=0)
            fig = get_scatter(c_df,dropdown_politicos.value,dropdown_temas.value)
            display(fig.show())
        else:
            display(print("Lo sentimos no hemos encontrado suficientes tweets para ejecutar el análisis"))
    with cluster_output:
        if(len(recuperados)>2):
            # Obtenemos gráfica del clúster y la enviamos a la salida (con el método display)
            fig = get_cluster(preprocesados)
            display(fig.show())
        else:
            display(print("Lo sentimos no hemos encontrado suficientes tweets para ejecutar el análisis"))
    with cloud_output:
        # Obtenemos gráfica de word cloud y la enviamos a la salida (con el método display)
        if(len(recuperados)>2):
            wordcloud = get_word_cloud(preprocesados,"pre_procesado")
            wordcloudEn =  get_word_cloud(preprocesados,"traducido")
            
            fig = plt.figure(figsize=(10,10))
            
            ax1 = fig.add_subplot(1,2,1)
            plt.imshow(wordcloud, interpolation='bilinear')
            plt.axis("off")
            plt.tight_layout(pad=0)
            
            ax2 = fig.add_subplot(1,2,2)
            plt.imshow(wordcloudEn, interpolation='bilinear')
            plt.axis("off")
            plt.tight_layout(pad=0)
        
            display(plt.show())
        else:
            display(print("Lo sentimos no hemos encontrado suficientes tweets para ejecutar el análisis"))
        
    with lda_output:
        # obtener gráfica de LDA y enviarla a la salida (con el método display)
        if(len(recuperados)>2):
            lda_display = get_lda(preprocesados)
            display(pyLDAvis.display(lda_display))
        else:
            display(print("Lo sentimos no hemos encontrado suficientes tweets para ejecutar el análisis"))

# asociar al botón el manejador definido        
btn.on_click(btn_eventhandler)

#Definir los tabs asociando cada una de las salidas 
tab = widgets.Tab([output_df, output_pre, output_sen, output_res, plot_output, cluster_output, cloud_output, lda_output], 
                  layout=widgets.Layout(width='100%', height='100%'))
tab.set_title(0, 'Conjunto')
tab.set_title(1, 'Pre - procesado')
tab.set_title(2, 'Sentimientos')
tab.set_title(3, 'Resumen')
tab.set_title(4, 'Conclusión gráfica')
tab.set_title(5, 'Clustering')
tab.set_title(6, 'Word Cloud')
tab.set_title(7, 'LDA')
#mostrar tabs
display(tab)
   

C:\Users\Usuario\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.

C:\Users\Usuario\anaconda3\lib\site-packages\ipywidgets\widgets\widget.py:412: DeprecationWarning:

Passing unrecognized arguments to super(IntText).__init__(min=100, max=5000).
object.__init__() takes exactly one argument (the instance to initialize)
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.

